<a href="https://colab.research.google.com/github/jlrp132/Sesimic_viewer/blob/main/seismic_displayer_ipywidget_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
!pip install segyio
import segyio

     |████████████████████████████████| 92kB 3.9MB/s 


In [4]:
# specify your path
norne = '/content/drive/MyDrive/Colab_Data/norne4d_2006-full-mini.sgy'

# choose which dataset to display
filename = norne

In [6]:
# read seismic data with segyio
with segyio.open(filename) as f:

  data = segyio.tools.cube(f)
  inline_data = f.iline
  crossline_data = f.xline

  inlines = f.ilines
  crosslines = f.xlines
  twt = f.samples
  sample_rate = segyio.tools.dt(f) / 1000
  print('Inline range from', inlines[0], 'to', inlines[-1])
  print('Crossline range from', crosslines[0], 'to', crosslines[-1])
  print('TWT from', twt[0], 'to', twt[-1])   
  print('Sample rate:', sample_rate, 'ms')

  clip_percentile = 99
  vm = np.percentile(data, clip_percentile)

f'The {clip_percentile}th percentile is {vm:.0f}; the max amplitude is {data.max():.0f}'

Inline range from 1000 to 1200
Crossline range from 2000 to 2300
TWT from 0.0 to 4000.0
Sample rate: 4.0 ms


'The 99th percentile is 2330; the max amplitude is 22711'

In [150]:
"""
Interactive seismic viewer

Input:

data, inlines, crosslines, twt, sample_rate, vm: output from segyio.read
data: the whole cube (3D numpy array)
inlines: inline locations (1D numpy array)
crosslines: crossline locations (1D numpy array)
twt: two-way travel time (1D numpy array)
sample_rate: sampling rate (float)
vm: 99th percentile of data

"""

from __future__ import print_function
from ipywidgets import interact, ToggleButtons, interactive_output
import ipywidgets as widgets
import segyio

# widgets
type_ = ToggleButtons(description = 'Selection', options = ['Inline','Crossline','Timeslice'])
cmap_button = ToggleButtons(description ='Colormaps', options = ['gray','seismic','RdBu','PuOr'])

inline_loc = widgets.IntSlider(min = min(inlines), max = max(inlines), step = 5, description = 'InLine')
xline_loc = widgets.IntSlider(min = min(crosslines), max = max(crosslines), step = 5, description = 'CrossLine')
timeslice_loc = widgets.IntSlider(min = min(twt), max = max(twt), step = sample_rate, description = 'TimeSlice')
timerange_loc = widgets.IntRangeSlider(min = min(twt), max = max(twt), step = sample_rate, orientation = 'horizontal', description = 'TimeRange')

# user interface
left_box = widgets.VBox([type_, inline_loc, xline_loc, timeslice_loc])
right_box = widgets.VBox([cmap_button, timerange_loc])

ui = widgets.HBox([left_box, right_box])


def seis_widget(type_ = type_, 
                inline_loc = inline_loc,
                xline_loc = xline_loc,
                timeslice_loc = timeslice_loc,
                timerange = timerange_loc,
                cmap = cmap_button):
  
  if type_ == 'Inline':

    with segyio.open(filename) as f: 
        inline_slice = f.iline[inline_loc][:, int(timerange_loc.value[0]/sample_rate):int(timerange_loc.value[1]/sample_rate)]

        plt.figure(figsize=(20, 10))
        plt.title('Seismic at Inline {}'.format(inline_loc))
        extent = [crosslines[0], crosslines[-1], twt[int(timerange_loc.value[1]/sample_rate)], twt[int(timerange_loc.value[0]/sample_rate)]]

        p1 = plt.imshow(inline_slice.T, cmap=cmap, aspect='auto', extent=extent,
                        vmin=-vm, vmax=vm, interpolation='bicubic')

        plt.xlabel('Crossline'); plt.ylabel('TWT')
        plt.colorbar()
        plt.show()

  if type_ == 'Crossline':

    with segyio.open(filename) as f:
        xline_slice = f.xline[xline_loc][:, int(timerange_loc.value[0]/sample_rate):int(timerange_loc.value[1]/sample_rate)]  

        plt.figure(figsize=(20, 10))
        plt.title('Seismic at Crossline {}'.format(xline_loc))
        extent = [inlines[0], inlines[-1], twt[int(timerange_loc.value[1]/sample_rate)], twt[int(timerange_loc.value[0]/sample_rate)]]

        p1 = plt.imshow(xline_slice.T, cmap=cmap, aspect='auto', extent=extent, 
                        vmin=-vm, vmax=vm, interpolation='bicubic')

        plt.xlabel('Inline'); plt.ylabel('TWT')
        plt.colorbar()
        plt.show()
  
  if type_ == 'Timeslice':

    id = np.where(twt == timeslice_loc)[0][0]
    tslice = data[:,:,id]

    plt.figure(figsize=(7,10))
    plt.title('Seismic at Timeslice {} ms'.format(timeslice_loc))
    extent = [inlines[0], inlines[-1], crosslines[-1], crosslines[0]]

    p1 = plt.imshow(tslice.T, cmap=cmap, aspect='auto', extent=extent, 
                    vmin=-vm, vmax=vm, interpolation='bicubic')


    plt.xlabel('Inline'); plt.ylabel('Crossline')
    plt.gca().xaxis.set_ticks_position('top') # axis on top
    plt.gca().xaxis.set_label_position('top') # label on top
    plt.xlim(min(inlines), max(inlines))
    plt.ylim(min(crosslines), max(crosslines))
    plt.axis('equal')
    plt.colorbar()
    plt.show()   

################################# Output #######################################
out = widgets.interactive_output(seis_widget, {'type_' : type_, 
                                                'inline_loc' : inline_loc,
                                                'xline_loc' : xline_loc,
                                                'timeslice_loc' : timeslice_loc,
                                                'timerange' : timerange_loc,
                                                'cmap' : cmap_button})
display(ui, out)

Output()